In [ ]:
# 시스템 및 파일 조작을 위한 라이브러리
import sys
import os
# 이미지 처리 및 수치 연산을 위한 라이브러리
import cv2
import numpy as np
# HTTP 요청을 위한 라이브러리
import requests
# GUI 구성을 위한 PyQt6 라이브러리
from PyQt6 import uic, QtGui
from PyQt6.QtWidgets import QApplication, QMainWindow
from PyQt6.QtCore import Qt, QThread, pyqtSignal

In [ ]:
# UI 파일 경로 및 백엔드 서버 URL 설정
UI_PATH = "./ui/mainwindow.ui"
BACKEND_URL = "http://192.168.0.24:8000"

In [4]:
class VideoThread(QThread):
    """
    백엔드 서버로부터 MJPEG 스트림을 비동기적으로 수신하는 스레드 클래스입니다.
    UI의 멈춤 현상을 방지하기 위해 별도의 스레드에서 네트워크 작업을 수행합니다.
    """
    # 메인 스레드(UI)로 디코딩된 이미지(numpy array)를 전달하기 위한 시그널 정의
    change_pixmap_signal = pyqtSignal(np.ndarray)

    def __init__(self, url):
        super().__init__()
        self._run_flag = True
        self.url = url

    def run(self):
        """스레드 실행 메인 루프: 스트림 연결 및 프레임 파싱"""
        try:
            # requests.get에 stream=True를 설정하여 전체 응답을 한 번에 받지 않고 스트리밍으로 받음
            # timeout=5는 연결 시도 시 5초간 응답이 없으면 예외를 발생시킴
            with requests.get(self.url, stream=True, timeout=5) as r:
                r.raise_for_status() # 200 OK가 아닌 경우 예외 발생
                
                # 수신된 데이터를 임시 저장할 버퍼
                byte_buffer = b''
                # 서버로부터 10KB 단위로 데이터를 읽어옴
                for chunk in r.iter_content(chunk_size=1024*10):
                    # 스레드 중지 플래그 확인
                    if not self._run_flag:
                        break
                        
                    # 읽어온 청크를 버퍼에 추가
                    byte_buffer += chunk
                    # JPEG 이미지의 시작(0xffd8)과 끝(0xffd9) 마커를 찾음
                    start = byte_buffer.find(b'\xff\xd8')
                    end = byte_buffer.find(b'\xff\xd9')
                    
                    # 시작과 끝 마커가 모두 존재하면 하나의 온전한 JPEG 프레임이 있는 것임
                    if start != -1 and end != -1:
                        # 버퍼에서 JPEG 데이터 추출 (끝 마커 2바이트 포함)
                        jpg = byte_buffer[start:end+2]
                        # 처리한 프레임 이후의 데이터만 버퍼에 남김
                        byte_buffer = byte_buffer[end+2:]
                        
                        try:
                            # 바이너리 데이터를 numpy 배열로 변환 후 OpenCV 이미지로 디코딩
                            frame = cv2.imdecode(np.frombuffer(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)
                            # 디코딩 성공 시 시그널을 통해 UI 스레드로 이미지 전달
                            if frame is not None:
                                self.change_pixmap_signal.emit(frame)
                        except Exception as e:
                            print(f"Frame decode error: {e}")

        except requests.exceptions.RequestException as e:
            print(f"Could not connect to backend: {e}")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            
        print("VideoThread finished.")

    def stop(self):
        """스레드 종료 요청: 플래그를 False로 설정하고 스레드가 종료될 때까지 대기"""
        self._run_flag = False
        self.wait()

class ClientWindow(QMainWindow):
    """메인 애플리케이션 윈도우 클래스"""
    def __init__(self):
        super().__init__()
        # .ui 파일이 실제로 존재하는지 확인
        if not os.path.exists(UI_PATH):
            print(f"Error: UI file not found at '{UI_PATH}'")
            sys.exit(1)

        # .ui 파일을 로드하여 현재 인스턴스(self)에 UI 요소를 연결
        uic.loadUi(UI_PATH, self)
        self.setWindowTitle("Detection Client")
        # 창을 최대화 상태로 표시
        self.showMaximized()

        # UI에 'log_output'이라는 이름의 위젯이 있다면 로그 메시지 출력
        if hasattr(self, "log_output"):
            # 로그의 최대 줄 수를 100줄로 제한
            self.log_output.setMaximumBlockCount(100)
            self.log_output.appendPlainText("Client started. Connecting to backend...")

        # 비디오 수신을 위한 백그라운드 스레드 생성
        self.thread = VideoThread(BACKEND_URL)
        # 스레드에서 이미지가 오면 update_image 메서드 호출 연결
        self.thread.change_pixmap_signal.connect(self.update_image)
        # 스레드 시작
        self.thread.start()

    def closeEvent(self, event):
        # 윈도우가 닫힐 때 호출되는 이벤트 핸들러
        print("Closing window...")
        # 비디오 스레드를 안전하게 종료
        self.thread.stop()
        event.accept()

    def update_image(self, cv_img):
        """VideoThread로부터 받은 OpenCV 이미지를 UI의 QLabel에 표시"""
        # OpenCV 형식(BGR)을 Qt 형식(QPixmap)으로 변환
        qt_img = self.convert_cv_qt(cv_img)
        
        # UI에 'video_display' 라벨이 있는 경우 이미지 설정
        if hasattr(self, "video_display"):
            # 라벨의 현재 크기에 맞춰 이미지를 스케일링 (비율 유지, 부드러운 변환)
            pixmap = qt_img.scaled(
                self.video_display.size(),
                Qt.AspectRatioMode.KeepAspectRatio,
                Qt.TransformationMode.SmoothTransformation
            )
            self.video_display.setPixmap(pixmap)

    def convert_cv_qt(self, cv_img):
        """OpenCV 이미지(numpy array)를 PyQt의 QPixmap으로 변환하는 헬퍼 메서드"""
        # OpenCV는 BGR 순서를 사용하므로 RGB 순서로 변환
        rgb_image = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
        # 이미지의 높이, 너비, 채널 수 가져오기
        h, w, ch = rgb_image.shape
        # 한 줄의 바이트 수 계산 (너비 * 채널 수)
        bytes_per_line = ch * w
        # QImage 객체 생성 (데이터, 너비, 높이, 라인당 바이트 수, 포맷)
        convert_to_Qt_format = QtGui.QImage(
            rgb_image.data, w, h, bytes_per_line, QtGui.QImage.Format.Format_RGB888
        )
        # QImage를 QPixmap으로 변환하여 반환
        return QtGui.QPixmap.fromImage(convert_to_Qt_format)

In [ ]:
# QApplication 인스턴스 생성 (커맨드 라인 인수 전달)
app = QApplication(sys.argv)
 # 메인 윈도우 생성 및 표시
window = ClientWindow()
window.show()
# 이벤트 루프 시작 및 종료 코드 반환
sys.exit(app.exec())